## Data Management

In [1]:
import pandas as pd
import numpy as np

The original dataframe "df" contains the date, distance, and runtime of each run from 2010-present. The goal of this data management process is to calculate a "pace" column from the distance and time of each run.

In [15]:
df = pd.read_excel("~/Documents/NIH IRTA/Running Log.xlsx")

In [90]:
pd.options.display.max_rows=20
print(df)

            Date  Mileage   Time   time_h
0     2009-04-25     3.10  29:38  0:29:38
1     2009-11-21     3.10  22:51  0:22:51
2     2010-01-04     1.30  17:00  0:17:00
3     2010-01-05     2.60  28:00  0:28:00
4     2010-01-07     2.60  28:00  0:28:00
5     2010-01-11     1.30    NaN      NaN
6     2010-01-14     2.60  26:00  0:26:00
7     2010-01-16     4.05  35:48  0:35:48
8     2010-01-19     2.60  24:00  0:24:00
9     2010-01-25     2.66  24:15  0:24:15
...          ...      ...    ...      ...
1398  2016-12-19     4.22  30:54  0:30:54
1399  2016-12-20     4.10  34:29  0:34:29
1400  2016-12-21     4.10  33:34  0:33:34
1401  2016-12-22     4.49  33:27  0:33:27
1402  2016-12-23     6.06  45:24  0:45:24
1403  2016-12-26     6.10  45:49  0:45:49
1404  2016-12-27     5.34  44:01  0:44:01
1405  2016-12-28     5.00  39:00  0:39:00
1406  2016-12-29     7.00  50:35  0:50:35
1407  2016-12-31     5.37  42:46  0:42:46

[1408 rows x 4 columns]


The main obstacle in creating a pace column is that it is not easy to do math with times in mm:ss format. So the "Time" column must be reformatted into minutes, with fractions of minutes put in decimal format (e.g. instead of "29:38", output = "29.633").

In [92]:
print(df.iloc[[0,1389]])

            Date  Mileage     Time   time_h
0     2009-04-25      3.1    29:38  0:29:38
1389  2016-11-27     10.0  1:10:51  1:10:51


However, as can be seen above, some of my runs lasted longer than an hour and were thus input in h:mm:ss format. When I attempted to delimit the col by ":", the "1" (hour) in row 1389 was put in the same column as the "29" (minutes) in row 0.

To work around this problem, I computed a new col "time_h" with a standardized time format of "h:mm:ss" as seen below.

In [93]:
# added new col to standardize format of time var as "h:mm:ss"
df["time_h"] = pd.np.where(df.Time.str.contains('\d:\d\d:\d\d', regex = True), df["Time"], "0:" + df["Time"])
print(df.iloc[[0,1389]])

            Date  Mileage     Time   time_h
0     2009-04-25      3.1    29:38  0:29:38
1389  2016-11-27     10.0  1:10:51  1:10:51


Then I created a new dataframe where df['time_h'] was delimited by ":" into three columns.

Next, I merged the "new" dataframe with the "df" dataframe and called the resulting dataframe "result".

Finally, in "result", I renamed the vars from "new" to be "hours", "minutes", and "seconds".

These changes can be seen in the following three sections of code.

In [95]:
# created new dataframe with "time_h" col expanded into hr, min, and sec vars
new = df["time_h"].str.split(pat = ":", expand = True)
print(new)

        0    1    2
0       0   29   38
1       0   22   51
2       0   17   00
3       0   28   00
4       0   28   00
5     NaN  NaN  NaN
6       0   26   00
7       0   35   48
8       0   24   00
9       0   24   15
...   ...  ...  ...
1398    0   30   54
1399    0   34   29
1400    0   33   34
1401    0   33   27
1402    0   45   24
1403    0   45   49
1404    0   44   01
1405    0   39   00
1406    0   50   35
1407    0   42   46

[1408 rows x 3 columns]


In [96]:
result = pd.concat([df, new], axis=1, sort=False)
print(result)

            Date  Mileage   Time   time_h    0    1    2
0     2009-04-25     3.10  29:38  0:29:38    0   29   38
1     2009-11-21     3.10  22:51  0:22:51    0   22   51
2     2010-01-04     1.30  17:00  0:17:00    0   17   00
3     2010-01-05     2.60  28:00  0:28:00    0   28   00
4     2010-01-07     2.60  28:00  0:28:00    0   28   00
5     2010-01-11     1.30    NaN      NaN  NaN  NaN  NaN
6     2010-01-14     2.60  26:00  0:26:00    0   26   00
7     2010-01-16     4.05  35:48  0:35:48    0   35   48
8     2010-01-19     2.60  24:00  0:24:00    0   24   00
9     2010-01-25     2.66  24:15  0:24:15    0   24   15
...          ...      ...    ...      ...  ...  ...  ...
1398  2016-12-19     4.22  30:54  0:30:54    0   30   54
1399  2016-12-20     4.10  34:29  0:34:29    0   34   29
1400  2016-12-21     4.10  33:34  0:33:34    0   33   34
1401  2016-12-22     4.49  33:27  0:33:27    0   33   27
1402  2016-12-23     6.06  45:24  0:45:24    0   45   24
1403  2016-12-26     6.10  45:4

In [97]:
df2 = result.rename(columns={"time_h": "time_corr", 0: "hours", 1: "minutes", 2: "seconds"})
print(df2)

            Date  Mileage   Time time_corr hours minutes seconds
0     2009-04-25     3.10  29:38   0:29:38     0      29      38
1     2009-11-21     3.10  22:51   0:22:51     0      22      51
2     2010-01-04     1.30  17:00   0:17:00     0      17      00
3     2010-01-05     2.60  28:00   0:28:00     0      28      00
4     2010-01-07     2.60  28:00   0:28:00     0      28      00
5     2010-01-11     1.30    NaN       NaN   NaN     NaN     NaN
6     2010-01-14     2.60  26:00   0:26:00     0      26      00
7     2010-01-16     4.05  35:48   0:35:48     0      35      48
8     2010-01-19     2.60  24:00   0:24:00     0      24      00
9     2010-01-25     2.66  24:15   0:24:15     0      24      15
...          ...      ...    ...       ...   ...     ...     ...
1398  2016-12-19     4.22  30:54   0:30:54     0      30      54
1399  2016-12-20     4.10  34:29   0:34:29     0      34      29
1400  2016-12-21     4.10  33:34   0:33:34     0      33      34
1401  2016-12-22     4.49

In [82]:
df2['hours'] = pd.to_numeric(df2['hours'])
df2['minutes'] = pd.to_numeric(df2['minutes'])
df2['seconds'] = pd.to_numeric(df2['seconds'])

In [83]:
time_seconds = df2.hours*3600 + df2.minutes*60 + df2.seconds
df2['time_s'] = time_seconds
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s
0     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0
1     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0
2     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0
3     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0
4     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0
5     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN
6     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0
7     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0
8     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0
9     2010-01-25     2.66    24:15   0:24:15    0.0     24.0     15.0  1455.0
10    2010-01-26     2.60    24:19   0:24:19    0.0     24.0     19.0  1459.0
11    2010-01-28     3.27    26:29   0:26:29    0.0     26.0    

In [84]:
df2['Mileage'] = pd.to_numeric(df2['Mileage'])
pace_s = df2.time_s / df2.Mileage
df2['pace_s'] = pace_s
print(df2.tail(10))

            Date  Mileage   Time time_corr  hours  minutes  seconds  time_s  \
1398  2016-12-19     4.22  30:54   0:30:54    0.0     30.0     54.0  1854.0   
1399  2016-12-20     4.10  34:29   0:34:29    0.0     34.0     29.0  2069.0   
1400  2016-12-21     4.10  33:34   0:33:34    0.0     33.0     34.0  2014.0   
1401  2016-12-22     4.49  33:27   0:33:27    0.0     33.0     27.0  2007.0   
1402  2016-12-23     6.06  45:24   0:45:24    0.0     45.0     24.0  2724.0   
1403  2016-12-26     6.10  45:49   0:45:49    0.0     45.0     49.0  2749.0   
1404  2016-12-27     5.34  44:01   0:44:01    0.0     44.0      1.0  2641.0   
1405  2016-12-28     5.00  39:00   0:39:00    0.0     39.0      0.0  2340.0   
1406  2016-12-29     7.00  50:35   0:50:35    0.0     50.0     35.0  3035.0   
1407  2016-12-31     5.37  42:46   0:42:46    0.0     42.0     46.0  2566.0   

          pace_s  
1398  439.336493  
1399  504.634146  
1400  491.219512  
1401  446.993318  
1402  449.504950  
1403  450.655738

In [85]:
pace_min = df2.pace_s / 60
df2['pace_min'] = pace_min
print(df2.tail(10))

            Date  Mileage   Time time_corr  hours  minutes  seconds  time_s  \
1398  2016-12-19     4.22  30:54   0:30:54    0.0     30.0     54.0  1854.0   
1399  2016-12-20     4.10  34:29   0:34:29    0.0     34.0     29.0  2069.0   
1400  2016-12-21     4.10  33:34   0:33:34    0.0     33.0     34.0  2014.0   
1401  2016-12-22     4.49  33:27   0:33:27    0.0     33.0     27.0  2007.0   
1402  2016-12-23     6.06  45:24   0:45:24    0.0     45.0     24.0  2724.0   
1403  2016-12-26     6.10  45:49   0:45:49    0.0     45.0     49.0  2749.0   
1404  2016-12-27     5.34  44:01   0:44:01    0.0     44.0      1.0  2641.0   
1405  2016-12-28     5.00  39:00   0:39:00    0.0     39.0      0.0  2340.0   
1406  2016-12-29     7.00  50:35   0:50:35    0.0     50.0     35.0  3035.0   
1407  2016-12-31     5.37  42:46   0:42:46    0.0     42.0     46.0  2566.0   

          pace_s  pace_min  
1398  439.336493  7.322275  
1399  504.634146  8.410569  
1400  491.219512  8.186992  
1401  446.9933

In [94]:
print(df.iloc[[0,800]])

                    Date  Mileage   Time   time_h
0             2009-04-25     3.10  29:38  0:29:38
800  2014-12-16 00:00:00     4.58  36:59  0:36:59


In [87]:
df2.Date = df2.Date.astype(str)
df2['Date'] = df2['Date'].str[:10]
print(df2)

            Date  Mileage     Time time_corr  hours  minutes  seconds  time_s  \
0     2009-04-25     3.10    29:38   0:29:38    0.0     29.0     38.0  1778.0   
1     2009-11-21     3.10    22:51   0:22:51    0.0     22.0     51.0  1371.0   
2     2010-01-04     1.30    17:00   0:17:00    0.0     17.0      0.0  1020.0   
3     2010-01-05     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
4     2010-01-07     2.60    28:00   0:28:00    0.0     28.0      0.0  1680.0   
5     2010-01-11     1.30      NaN       NaN    NaN      NaN      NaN     NaN   
6     2010-01-14     2.60    26:00   0:26:00    0.0     26.0      0.0  1560.0   
7     2010-01-16     4.05    35:48   0:35:48    0.0     35.0     48.0  2148.0   
8     2010-01-19     2.60    24:00   0:24:00    0.0     24.0      0.0  1440.0   
9     2010-01-25     2.66    24:15   0:24:15    0.0     24.0     15.0  1455.0   
10    2010-01-26     2.60    24:19   0:24:19    0.0     24.0     19.0  1459.0   
11    2010-01-28     3.27   

In [88]:
df3 = df2[['Date','Mileage','time_corr','pace_min']]
pd.options.display.max_rows=2000
print(df3)
pd.options.display.max_rows=20

            Date  Mileage time_corr   pace_min
0     2009-04-25     3.10   0:29:38   9.559140
1     2009-11-21     3.10   0:22:51   7.370968
2     2010-01-04     1.30   0:17:00  13.076923
3     2010-01-05     2.60   0:28:00  10.769231
4     2010-01-07     2.60   0:28:00  10.769231
5     2010-01-11     1.30       NaN        NaN
6     2010-01-14     2.60   0:26:00  10.000000
7     2010-01-16     4.05   0:35:48   8.839506
8     2010-01-19     2.60   0:24:00   9.230769
9     2010-01-25     2.66   0:24:15   9.116541
10    2010-01-26     2.60   0:24:19   9.352564
11    2010-01-28     3.27   0:26:29   8.098879
12    2010-02-04     2.60   0:24:03   9.250000
13    2010-02-11     2.60       NaN        NaN
14    2010-02-16     2.60   0:22:04   8.487179
15    2010-02-18     2.60   0:22:56   8.820513
16    2010-03-13     3.00   0:29:28   9.822222
17    2010-03-15     3.00   0:28:00   9.333333
18    2010-03-16     3.00   0:30:00  10.000000
19    2010-04-04     2.60   0:22:02   8.474359
20    2010-04

In [89]:
df3.to_csv("clean.csv", index=False)